In [8]:
from dataset.read_csv import review_data
from transformers import BertForSequenceClassification, logging
from models.sentiment_classifier import SentimentClassifier
from torch.utils.data import DataLoader
from utils import  test_data_tokenizer, label_interpreter


import os
import torch
import tqdm
import random
import numpy as np
import torch.nn as nn
logging.set_verbosity_warning()


<H1> Load the model </h1>

3 models are available for testing.

"Full Azerbaijani" has been trained on nearly 48k data. (remaining 2k data has been lost due unstable internet connection during translation using google translate API)

Other models are trained on 21k data to get the results faster


Time spent for translation algorithm: 24 hours

21k dataset training: 38 mins/epoch


48k dataset training: 1h 18 mins / epoch



In [9]:
available_models = {"Azerbaijani":"models/latest_model_aze.pth",
                    "English": "models/latest_model.pth",
                    "Full Azerbaijani": "models/latest_model_aze_full.pth"}


test_data = review_data("test")
model = SentimentClassifier(2)


print('load model from %s ...' % available_models["Full Azerbaijani"])
model.load_state_dict(torch.load(available_models["Full Azerbaijani"])['state_dict'])
print('Done!')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


load model from models/latest_model_aze_full.pth ...
Done!


<h2> Test single sentence </h2>

In [14]:
review = "Bu artikl haqqında fikirlərim yaxşıdır. şiddət yoxdur. Hamıya məsləhət görürəm"
model.eval().cpu()

input_ids, attention_masks = test_data_tokenizer(review)
outputs = model(input_ids= input_ids.unsqueeze(dim= 0),
                            attention_mask = attention_masks.unsqueeze(dim = 0))

_, preds = torch.max(outputs, dim=1)

label_interpreter(preds)

positive


<h2> Batch Testing with test data </h2>

In [15]:
dataloader_test = DataLoader(
    test_data,
    batch_size=1
)

print("if cuda is working 1 min. if not around 10 mins)")
tq = tqdm.tqdm(total = len(dataloader_test))
tq.set_description('Test')

test_iterator = enumerate(dataloader_test)
correct_predictions=0
model.eval().cuda()

for _, batch in test_iterator:
    with torch.no_grad():
        input_ids, attention_masks, label = batch

        outputs = model(input_ids= input_ids.cuda(),
                                attention_mask = attention_masks.cuda())
                
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == label.cuda())
                
        tq.update()

tq.close()

print("Accuracy: ", correct_predictions/len(dataloader_test))



if cuda is working 1 min. if not around 10 mins)


Test: 100%|██████████| 1961/1961 [01:09<00:00, 28.29it/s]

Accuracy:  tensor(0.8445, device='cuda:0')
